In [1]:
## This is a test file to train to track an object
## Author : Avadesh Meduri
## Date : 8/04/2022

from matplotlib import pyplot as plt
import numpy as np

from torch.utils.data import Dataset
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision.io import read_image
from torch.utils.data import DataLoader, RandomSampler, Sampler

import cv2

In [6]:
class BoxDataSet(Dataset):
    
    def __init__(self, fname):
        
        self.img_dir = "./image_data/" + fname
        self.data = np.load("./position_data/" + fname + ".npz")
        self.y_train = torch.tensor(self.data["position"])
    
    def __len__(self):
        return len(self.y_train)
    
    def __getitem__(self, idx):
    
        image = cv2.imread(self.img_dir + "/" + str(idx) + ".png", cv2.IMREAD_UNCHANGED )
        label = self.y_train[idx]
        
        return torch.tensor(image.astype(np.int64)), label
        
        

In [20]:
class C_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(4, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(293904, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
#         print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [21]:
net = C_Net()

lr = 1e-4
eps = int(4e4)
batch_size = 32

optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss = torch.nn.MSELoss() #torch.nn.MSELoss() #torch.nn.HuberLoss()
dl = DataLoader(BoxDataSet("data1"), batch_size = 128)

for i in range(eps):
    
    for x_train_batch, y_train_batch in dl:
        y_pred = net(x_train_batch)
        error = loss(y_pred, y_train_batch) 
        print("The iteration number : " + str(i) + " The loss is :" + str(error.cpu().detach().numpy()), end='\r', flush  = True)
        optimizer.zero_grad()
        error.backward()
        optimizer.step()

RuntimeError: Given groups=1, weight of size [6, 3, 5, 5], expected input[101, 480, 640, 4] to have 3 channels, but got 480 channels instead

In [ ]:
# import time

# # for i in range(1000):
# with torch.no_grad():
#     tmp = x_train[0][None,:,:,:]
#     %timeit net(tmp)
